<a href="https://colab.research.google.com/github/lkarjun/Data-Science-from-Scratch/blob/master/17%20Decision%20Tree/decisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Tree

## Entropy

In [2]:
from typing import List
import math

In [3]:
def entropy(class_probabilities: List[float]) -> float:
    '''given a list of class probabilities, compute the entropy'''
    return sum(-p * math.log(p, 2)
                for p in class_probabilities
                if p > 0)

In [4]:
assert entropy([0.5, 0.5]) == 1

In [5]:
from typing import Any
from collections import Counter

In [6]:
def class_probabilities(labels: List[Any]) -> List[float]:
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities(labels))    

In [ ]:
assert data_entropy(['a']) == 0
assert data_entropy([True, True]) == 0.0
assert data_entropy(['yes', 'yes', 'no', 'no']) == 1

## The Entropy of a Partition

In [24]:
def partion_entropy(subsets: List[List[Any]]) -> float:
    '''return the entropy from partition of data into subsets'''
    total_count = sum(len(subset) for subset in subsets)
    return sum(data_entropy(subset) * len(subset) / total_count
              for subset in subsets)

## Creating a Decision Tree

In [25]:
from typing import NamedTuple, Optional

In [26]:
class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None

In [27]:
input = [Candidate('Senior', 'Java', False, False, False),
         Candidate('Senior', 'Java', False, True, False),
         Candidate('Mid', 'Python', False, False, True),
         Candidate('Junior', 'Python', False, False, True),
         Candidate('Junior', 'R', True, False, True),
         Candidate('Junior', 'R', True, True, False),
         Candidate('Mid', 'R', True, True, True),
         Candidate('Senior', 'Python', False, False, False),
         Candidate('Senior', 'R', True, False, True),
         Candidate('Junior', 'Python', True, False, True),
         Candidate('Senior', 'Python', True, True, True),
         Candidate('Mid', 'Python', False, True, True),
         Candidate('Mid', 'Java', True, False, True),
         Candidate('Junior', 'Python', False, True, False)]